In [7]:
# ===================================================================================
# 1. CONFIGURAÇÃO DO AMBIENTE E IMPORTAÇÕES
# ===================================================================================
!pip install splink -q

import duckdb
import pandas as pd
from splink import Linker, SettingsCreator, DuckDBAPI
import splink.comparison_library as cl
from splink import block_on
from splink.exploratory import completeness_chart, profile_columns
from splink.blocking_analysis import cumulative_comparisons_to_be_scored_from_blocking_rules_chart, n_largest_blocks

print("Ambiente configurado com sucesso.")

Ambiente configurado com sucesso.


In [8]:
# ===================================================================================
# 2. CONEXÃO, PRÉ-PROCESSAMENTO E AMOSTRAGEM
# ===================================================================================
path_dados = 'dados_catalogo/processed/dados_consolidados_filtrados.parquet'
con = duckdb.connect(database=':memory:', read_only=False)
db_api = DuckDBAPI(connection=con)

# SQL para criar a VIEW com o dataset completo e colunas de array
sql_completo = f"""
CREATE OR REPLACE VIEW dados_completos AS
SELECT
    *,
    list_distinct(list_sort(string_to_array(trim(BB_Cast), ','))) as BB_Cast_array,
    list_distinct(list_sort(string_to_array(trim(BB_Directors), ','))) as BB_Directors_array
FROM read_parquet('{path_dados}');
"""
con.execute(sql_completo)
print("View 'dados_completos' (15 milhões de linhas) criada.")

# SQL para criar a VIEW com uma amostra de 200 mil linhas para exploração
sql_amostra = """
CREATE OR REPLACE VIEW dados_amostra AS
SELECT * FROM dados_completos USING SAMPLE 200000 ROWS;
"""
con.execute(sql_amostra)
print("View 'dados_amostra' (200 mil linhas) criada para exploração rápida.")

# Verificando a contagem de linhas em cada view
total_rows = con.execute("SELECT COUNT(*) FROM dados_completos").fetchone()[0]
sample_rows = con.execute("SELECT COUNT(*) FROM dados_amostra").fetchone()[0]
print(f"\nVerificação -> Linhas no dataset completo: {total_rows:,} | Linhas na amostra: {sample_rows:,}")

View 'dados_completos' (15 milhões de linhas) criada.
View 'dados_amostra' (200 mil linhas) criada para exploração rápida.

Verificação -> Linhas no dataset completo: 14,806,386 | Linhas na amostra: 200,000


In [15]:
# ===================================================================================
# CÉLULA 3: CONFIGURAÇÃO DO MODELO COM REGRA DE BLOQUEIO ESTRITA
# ===================================================================================
# Usando a regra de bloqueio simultânea para Título, Ano e Diretores, como solicitado.
# Esta regra é muito mais restritiva e eficiente.
settings = SettingsCreator(
    link_type="dedupe_only",
    unique_id_column_name="BB_UID",
    
    blocking_rules_to_generate_predictions=[
        block_on("BB_Title", "BB_Year", "BB_Directors"),
    ],
    
    comparisons=[
        cl.JaroWinklerAtThresholds("BB_Title", [0.9, 0.7]),
        cl.ExactMatch("BB_Year"),
        cl.ArrayIntersectAtSizes("BB_Directors_array", [1]),
        cl.ArrayIntersectAtSizes("BB_Cast_array", [3, 1]),
    ]
)
print("Configurações do modelo Splink criadas.")


Configurações do modelo Splink criadas.


In [10]:
print("\n--- Perfil da Coluna BB_Title (executando na amostra) ---")
profile_columns("dados_amostra", column_expressions=["BB_Title"], db_api=db_api)


--- Perfil da Coluna BB_Title (executando na amostra) ---


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.VConcatChart(...)

In [ ]:
# ===================================================================================
# CÉLULA 4: TREINAMENTO E PREDIÇÃO
# ===================================================================================
# Instanciamos o Linker e a API do DuckDB
db_api = DuckDBAPI(connection=con)
linker = Linker("dados_completos", settings, db_api=db_api)

# Treinamento dos parâmetros 'u' (ocorrências em pares aleatórios)
print("\nIniciando estimação das probabilidades 'u'...")
linker.training.estimate_u_using_random_sampling(max_pairs=5e6)

# Treinamento dos parâmetros 'm' (ocorrências em pares correspondentes)
# Usamos regras de bloqueio estritas para um treinamento eficiente
print("\nIniciando 1ª sessão de treinamento EM...")
linker.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Title", "BB_Year"))

print("\nIniciando 2ª sessão de treinamento EM...")
linker.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Directors", "BB_Year"))

print("\nTreinamento concluído. Gerando o gráfico de pesos...")
display(linker.visualisations.match_weights_chart())

# Predição e Clusterização
print("\nExecutando a predição de duplicatas...")
df_predict = linker.inference.predict()

print("\nAgrupando os resultados em clusters...")
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=0.95)


----- Estimating u probabilities using random sampling -----



Iniciando estimação das probabilidades 'u'...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - BB_Title (no m values are trained).
    - BB_Year (no m values are trained).
    - BB_Directors_array (no m values are trained).
    - BB_Cast_array (no m values are trained).



Iniciando 1ª sessão de treinamento EM...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."BB_Title" = r."BB_Title") AND (l."BB_Year" = r."BB_Year")

Parameter estimates will be made for the following comparison(s):
    - BB_Directors_array
    - BB_Cast_array

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - BB_Title
    - BB_Year


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
# Investigando os maiores blocos na amostra para a regra "BB_Title"
print("\n--- Maiores Blocos para 'block_on(\"BB_Title\")' (na amostra) ---")
n_largest_blocks(
    "dados_amostra",
    block_on("BB_Title"),
    db_api=db_api,
    link_type="dedupe_only",
    n_largest=10
).as_pandas_dataframe()

Interpretação da Análise Exploratória:

O completeness_chart nos mostrará se alguma coluna-chave tem muitos dados faltantes, o que diminuiria sua utilidade.

O profile_columns para BB_Title provavelmente revelará uma alta cardinalidade (muitos títulos únicos), mas também pode mostrar uma "cauda longa" de títulos muito frequentes (skew). Para BB_Year, veremos a distribuição de lançamentos ao longo do tempo.

Célula 3: Análise e Seleção de Regras de Bloqueio
Esta é uma etapa crítica para a performance. Vamos analisar diferentes regras de bloqueio para encontrar um conjunto que reduza o número de comparações a um nível gerenciável, sem perder muitos matches verdadeiros.

In [ ]:
# ===================================================================================
# 5. CONFIGURAÇÃO DO MODELO SPLINK
# ===================================================================================
settings = SettingsCreator(
    link_type="dedupe_only",
    unique_id_column_name="BB_UID",
    blocking_rules_to_generate_predictions=[
        block_on("substr(BB_Title, 1, 3)", "BB_Year"),
        block_on("substr(BB_Directors, 1, 5)", "BB_Year"),
    ],
    comparisons=[
        cl.JaroWinklerAtThresholds("BB_Title", [0.9, 0.7]),
        cl.ExactMatch("BB_Year"),
        cl.ArrayIntersectAtSizes("BB_Cast_array", [3, 1]),
        cl.ArrayIntersectAtSizes("BB_Directors_array", [1]),
    ],
    retain_intermediate_calculation_columns=True,
    retain_matching_columns=True,
)
print("Configuração (settings) do Splink criada.")

In [ ]:
# ===================================================================================
# 6. TREINAMENTO DO MODELO (NO DATASET COMPLETO)
# ===================================================================================
# IMPORTANTE: Agora instanciamos o Linker com a view do dataset COMPLETO
print("Iniciando o Linker no dataset completo de 15 milhões de linhas...")
linker = Linker("dados_completos", settings, db_api=db_api)

# A estimação de 'u' pode ser demorada. Usamos um número grande de pares para precisão.
print("\n--- Estimando probabilidades 'u' no dataset completo... ---")
linker.training.estimate_u_using_random_sampling(max_pairs=10e6) # 10 milhões de pares

# O treinamento EM usa regras de bloqueio restritas para ser eficiente
print("\n--- Iniciando 1ª sessão de treinamento EM (bloqueando por Título e Ano) ---")
linker.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Title", "BB_Year"))

print("\n--- Iniciando 2ª sessão de treinamento EM (bloqueando por Diretores e Ano) ---")
linker.training.estimate_parameters_using_expectation_maximisation(block_on("BB_Directors", "BB_Year"))

print("\nTreinamento do modelo concluído!")

In [ ]:
# ===================================================================================
# 7. ANÁLISE DO MODELO E PREDIÇÃO FINAL
# ===================================================================================
print("--- Gráfico de Pesos de Correspondência (Match Weights) do modelo final ---")
linker.visualisations.match_weights_chart()

In [ ]:
# Executa a predição no dataset completo
print("\n--- Executando a predição de duplicatas no dataset completo ---")
df_predict = linker.inference.predict()

# Agrupa os resultados em clusters para gerar a tabela de vínculos
print("\n--- Gerando a Tabela de Vínculos (ID Canônico) ---")
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=0.95)

print("\nAmostra da Tabela de Vínculos (BB_UID -> cluster_id):")
display(clusters.as_pandas_dataframe(limit=20))

In [ ]:
import os
import platform
import time

# Tempo em segundos antes de hibernar (aqui: 60 segundos = 1 minuto)
tempo_espera = 60

# Só executa se estiver no Windows
if platform.system() == "Windows":
    print(f"O computador vai hibernar em {tempo_espera} segundos...")
    time.sleep(tempo_espera)
    os.system("shutdown /h")
else:
    print("Este comando de hibernação só funciona no Windows.")
